In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
import tensorflow as tf

# Load the integer sequence the IMDB dataset with Keras.
index_offset = 3  # word index offset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=1000,
                                                      index_from=index_offset)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
# Prepare the dictionary of index to word.
word_to_id = imdb.get_word_index()
word_to_id = {k: (v + index_offset) for k, v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
id_to_word = {value: key for key, value in word_to_id.items()}
# Convert the word indices to words.
x_train = list(map(lambda sentence: ' '.join(
    id_to_word[i] for i in sentence), x_train))
x_test = list(map(lambda sentence: ' '.join(
    id_to_word[i] for i in sentence), x_test))
x_train = np.array(x_train, dtype=np.str)
x_test = np.array(x_test, dtype=np.str)
print(x_train.shape)  # (25000,)
print(y_train.shape)  # (25000, 1)
print(x_train[0][:50])  # <START> this film was just brilliant casting <UNK>

(25000,)
(25000, 1)
<START> this film was just brilliant casting <UNK>


In [1]:
import time
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import random
import pandas as pd
import os
Drugdata = []
druganwser = []
EPAdata = []
epaanwser = []
with open("../logP_labels.smi", "r") as EPA:
    with open("../alogptest.csv", "r") as DRUG:
        dlines = DRUG.readlines()
        Elines = EPA.readlines()
        for dline in dlines:
            dsl = dline.split()
            Drugdata.append(" ".join(dsl[0]))
            druganwser.append(dsl[1])
        for Eline in Elines:
            Esl = Eline.split()
            EPAdata.append(" ".join(Esl[0]))
            epaanwser.append(Esl[1])


x_train = np.array(EPAdata, dtype=np.str)
y_train = np.array(epaanwser, dtype=np.str)
x_test = np.array(Drugdata, dtype=np.str)
y_test = np.array(druganwser, dtype=np.str)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
MAX_LENGTH = 400
SOS_token = 1
EOS_token = 2
PAD_token = 0


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "PAD", 1: "SOS", 2: "EOS"}
        self.n_words = 3  # PAD 포함
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word

            self.n_words += 1
        else:
            self.word2count[word] += 1


EPAlang = Lang("EPA")
Druglang = Lang("Drug")
E = []
D = []


def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence_encond(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    while len(indexes) >= MAX_LENGTH-1:
        del indexes[MAX_LENGTH - 2]
    
    indexes.append(EOS_token)
    while len(indexes) < MAX_LENGTH:
        indexes.append(PAD_token)
    return indexes


def tensorsFromPair(EP):
    input_tensor = tensorFromSentence_encond(EPAlang, EP)
    return input_tensor


for EP in x_train:
    EPAlang.addSentence(EP)
    Druglang.addSentence(EP)
    ab = tensorsFromPair(EP)
    a = [str(ab[i]) for i in range(len(ab))]
    E.append(" ".join(a))
for Dru in x_test:
    EPAlang.addSentence(Dru)
    Druglang.addSentence(Dru)
    ab = tensorsFromPair(Dru)
    a = [str(ab[i]) for i in range(len(ab))]
    D.append(" ".join(a))

In [6]:
import tensorflow as tf
def correlation_coefficient(y_true, y_pred):
    pearson_r, update_op = tf.contrib.metrics.streaming_pearson_correlation(y_pred, y_true)
    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'correlation_coefficient' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        pearson_r = tf.identity(pearson_r)
        return pearson_r

In [2]:

x_train = np.array(E, dtype=np.str)
x_test = np.array(D, dtype=np.str)

print(x_train)

['3 3 4 3 3 5 3 6 3 3 3 7 8 3 3 9 8 10 11 3 8 3 3 7 9 3 11 3 6 9 12 11 3 9 10 11 3 3 5 9 3 11 3 4 9 10 11 3 9 8 10 11 3 10 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0'
 '3 3 9 8 10 11 10 3 3 9 8 10 11 3 4 9 10 11 3 3 3 5 3 6 3 3 3 7 8 3 3 9 8 10 11 3 3 3 7 9 3 11 3 6 3 9 10 11 3 3 5 4 3 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [7]:
import autokeras as ak

# Initialize the text classifier.
clf = ak.TextRegressor(max_trials=200, overwrite=False, metrics="correlation_coefficient")
# Feed the text classifier with training data.
clf.fit(x_train, y_train)
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

INFO:tensorflow:Reloading Oracle from existing project ./text_regressor/oracle.json
INFO:tensorflow:Reloading Tuner from ./text_regressor/tuner0.json
INFO:tensorflow:Oracle triggered exit


Traceback (most recent call last):
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 65, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/autokeras/graph.py", line 301, in build
    return self._compile_keras_model(hp, model)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/autokeras/graph.py", line 327, in _compile_keras_model
    loss=self._get_loss())
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/base.py", line 457, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/tr

Traceback (most recent call last):
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 65, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/autokeras/graph.py", line 301, in build
    return self._compile_keras_model(hp, model)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/autokeras/graph.py", line 327, in _compile_keras_model
    loss=self._get_loss())
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/base.py", line 457, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/tr

Traceback (most recent call last):
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 65, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/autokeras/graph.py", line 301, in build
    return self._compile_keras_model(hp, model)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/autokeras/graph.py", line 327, in _compile_keras_model
    loss=self._get_loss())
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/base.py", line 457, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/tr

Traceback (most recent call last):
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 65, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/autokeras/graph.py", line 301, in build
    return self._compile_keras_model(hp, model)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/autokeras/graph.py", line 327, in _compile_keras_model
    loss=self._get_loss())
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/base.py", line 457, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/tr

Traceback (most recent call last):
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 65, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/autokeras/graph.py", line 301, in build
    return self._compile_keras_model(hp, model)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/autokeras/graph.py", line 327, in _compile_keras_model
    loss=self._get_loss())
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/base.py", line 457, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/tr

Traceback (most recent call last):
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 65, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/autokeras/graph.py", line 301, in build
    return self._compile_keras_model(hp, model)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/autokeras/graph.py", line 327, in _compile_keras_model
    loss=self._get_loss())
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/base.py", line 457, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/junsu/anaconda3/envs/autokeras/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/tr

RuntimeError: Too many failed attempts to build model.

In [5]:
clf.export_model()